# Simple agent system for PDF documents

This notebook demonstrates how to create a simple chatbot that can ethier generate images or answer questions.

1. Load the necessary libraries and environment variables.
2. Create the textual chatbot.
3. Create the image generation chatbot.
4. Ask the user for input and generate a response.


Please try to not run this notebook to many time for the thing as each image generation cost some credits.

## Prerequisites

Before running this notebook, you need:
- An OpenAI API key

In [ ]:
# Install required packages
!pip install llama-index llama-index-llms-openai python-dotenv openai nest-asyncio nbconvert requests

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    "llama_index": "llama-index core",
    "llama_index.llms.openai": "llama-index-llms-openai",
    "dotenv": "python-dotenv",
    "openai": "OpenAI API",
    "nest_asyncio": "nest-asyncio", 
    "nbconvert": "nbconvert",
    "requests": "requests",
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

In [ ]:
import os
from dotenv import load_dotenv
import nest_asyncio

# Apply nest_asyncio to allow nested event loops (needed for some async operations)
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables or set them directly
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If environment variables are not loaded, you can set them here
# OPENAI_API_KEY = "your-openai-api-key"

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""

# Verify API key is set
if not OPENAI_API_KEY:
    print("⚠️ Warning: OPENAI_API_KEY is not set")
else:
    print("✅ API key is set")

# Configure the 2 LLMs

Setup the OpenAI language model to be use by our LLMs.

## Textual LLM

In [ ]:
from llama_index.llms.openai import OpenAI

# Initialize the textual LLM with LlamaIndex library for OpenAI
# Waring : Please not change the model name to avoid errors or extreme costs
textualChatbot = OpenAI(model="gpt-4.1-nano", api_key=OPENAI_API_KEY)

# Verify LLM setup with a simple test
response = textualChatbot.complete("Hello, I am a language model. ")
print("LLM Test Response:", response.text)

## Image chatbot

This section creates a chatbot that can generate images based on user input. It uses the OpenAI API to generate images and display them in the notebook.

This section is not build with LlamaIndex, but used the default OpenAI API to generate images. It is a simple chatbot that can generate images based on user input. It uses the OpenAI API to generate images and display them in the notebook.

In [ ]:
from openai import OpenAI
import os
import datetime
import base64
from IPython.display import Image, display

# Create an OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Step 1: Create the prompt as a variable
prompt = (
    "Create an artistic illustration of OpenAI and LlamaIndex working together. "
    "The OpenAI logo (a hexagonal knot) is connected with digital circuits to a "
    "stylized llama representing LlamaIndex, all in a futuristic AI research lab."
)

# Step 2: Generate image using OpenAI GPT-Image-1
print("Generating image with GPT-Image-1...")
image_response = client.images.generate(
    model="gpt-image-1",
    prompt=prompt,
    n=1,
    quality="medium",
    size="1024x1024",
)

# Step 3: Get the base64 encoded image and decode it
image_base64 = image_response.data[0].b64_json
image_bytes = base64.b64decode(image_base64)

# Step 4: Save the image to a file with timestamp
# Create timestamp for filename
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
image_dir = "image"
os.makedirs(image_dir, exist_ok=True)  # Create the image directory if it doesn't exist
image_path = os.path.join(image_dir, f"prompt-{timestamp}.png")

# Save the image directly from the bytes
with open(image_path, "wb") as f:
    f.write(image_bytes)
print(f"Image saved to {image_path}")

# Display the image in the notebook if in an IPython environment
try:
    display(Image(image_path))
except ImportError:
    print("IPython not available for displaying the image")


# Combine the 2 LLMs with a basic ask to the user


In [ ]:
import os
import datetime
import base64
from openai import OpenAI

# Step 1: Ask for the mode of the chatbot to the user
def get_chatbot_mode():
    mode = input("Enter the mode of the chatbot (textual/visual): ").strip().lower()
    if mode not in ["textual", "visual"]:
        print("Invalid mode. Please enter 'textual' or 'visual'.")
        return get_chatbot_mode()
    return mode
chatbot_mode = get_chatbot_mode()

# Step 2: Launch the desired chatbot mode based on user input
if chatbot_mode == "textual":
    # Get the prompt from the user
    prompt = input("Enter the prompt for the chatbot: ")
    # perform the query
    response = textualChatbot.complete(prompt)
    # Print the response
    print("Chatbot Response:", response.text)
    
elif chatbot_mode == "visual":
    # Create an OpenAI client
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # Get the prompt from the user
    prompt = input("Enter the prompt for the visual chatbot: ")
    
    # Generate image using OpenAI GPT-Image-1
    print("Generating image with GPT-Image-1...")
    image_response = client.images.generate(
        model="gpt-image-1",
        prompt=prompt,
        n=1,
        quality="low",
        size="1024x1024",
    )
    
    # Get the base64 encoded image and decode it
    image_base64 = image_response.data[0].b64_json
    image_bytes = base64.b64decode(image_base64)
    
    # Create timestamp for filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    image_dir = "image"
    os.makedirs(image_dir, exist_ok=True)
    
    # Create the image path
    image_path = os.path.join(image_dir, f"prompt-{timestamp}.png")
    
    # Save the image directly from the bytes
    with open(image_path, "wb") as f:
        f.write(image_bytes)
    print(f"Image saved to {image_path}")
    
    # Display the image in the notebook if in an IPython environment
    try:
        from IPython.display import Image, display
        display(Image(image_path))
    except ImportError:
        print("IPython not available for displaying the image")

